# Quantization of Text Embedding model from Sentence Transformers library

In [ ]:
%pip install optimum[openvino]
%pip install evaluate

## Quantize staticly model to 8-bit with NNCF via Optimum-Intel API

In [1]:
from functools import partial
import datasets
from transformers import AutoTokenizer
from optimum.intel import OVModelForFeatureExtraction, OVQuantizer, OVQuantizationConfig, OVConfig

MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"

model = OVModelForFeatureExtraction.from_pretrained(MODEL_ID)
model.save_pretrained("all-MiniLM-L6-v2")

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
DATASET_NAME = "squad"
dataset = datasets.load_dataset(DATASET_NAME)
int8_ptq_model_path = "all-MiniLM-L6-v2_int8"

quantizer = OVQuantizer.from_pretrained(model)


def preprocess_function(examples, tokenizer):
    return tokenizer(examples["sentence"], padding="max_length", max_length=384, truncation=True)


calibration_dataset = quantizer.get_calibration_dataset(
    "glue",
    dataset_config_name="sst2",
    preprocess_function=partial(preprocess_function, tokenizer=tokenizer),
    num_samples=300,
    dataset_split="train",
)

ov_config = OVConfig(quantization_config=OVQuantizationConfig())

quantizer.quantize(ov_config=ov_config, calibration_dataset=calibration_dataset, save_directory=int8_ptq_model_path)
tokenizer.save_pretrained(int8_ptq_model_path)

No OpenVINO files were found for sentence-transformers/all-MiniLM-L6-v2, setting `export=True` to convert the model to the OpenVINO IR. Don't forget to save the resulting model with `.save_pretrained()`
Framework not specified. Using pt to export the model.
Using framework PyTorch: 2.4.1+cpu
Overriding 1 configuration item(s)
	- use_cache -> False
Compiling the model to CPU ...


Output()

Output()

Output()

Output()

Configuration saved in all-MiniLM-L6-v2_int8/openvino_config.json


('all-MiniLM-L6-v2_int8/tokenizer_config.json',
 'all-MiniLM-L6-v2_int8/special_tokens_map.json',
 'all-MiniLM-L6-v2_int8/vocab.txt',
 'all-MiniLM-L6-v2_int8/added_tokens.json',
 'all-MiniLM-L6-v2_int8/tokenizer.json')

## Benchmark model accuracy on GLUE STSB task

In [2]:
from transformers import Pipeline
import torch.nn.functional as F
import torch


# copied from the model card
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


class SentenceEmbeddingPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        # we don"t have any hyperameters to sanitize
        preprocess_kwargs = {}
        return preprocess_kwargs, {}, {}

    def preprocess(self, inputs):
        encoded_inputs = self.tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")
        return encoded_inputs

    def _forward(self, model_inputs):
        outputs = self.model(**model_inputs)
        return {"outputs": outputs, "attention_mask": model_inputs["attention_mask"]}

    def postprocess(self, model_outputs):
        # Perform pooling
        sentence_embeddings = mean_pooling(model_outputs["outputs"], model_outputs["attention_mask"])
        # Normalize embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings

In [3]:
model = OVModelForFeatureExtraction.from_pretrained(MODEL_ID)
vanilla_emb = SentenceEmbeddingPipeline(model=model, tokenizer=tokenizer)

q_model = OVModelForFeatureExtraction.from_pretrained(int8_ptq_model_path)
q8_emb = SentenceEmbeddingPipeline(model=q_model, tokenizer=tokenizer)

No OpenVINO files were found for sentence-transformers/all-MiniLM-L6-v2, setting `export=True` to convert the model to the OpenVINO IR. Don't forget to save the resulting model with `.save_pretrained()`
Framework not specified. Using pt to export the model.
Using framework PyTorch: 2.4.1+cpu
Overriding 1 configuration item(s)
	- use_cache -> False
Compiling the model to CPU ...
Compiling the model to CPU ...


In [4]:
from datasets import load_dataset
from evaluate import load

eval_dataset = load_dataset("glue", "stsb", split="validation")
metric = load("glue", "stsb")

In [5]:
def compute_sentence_similarity(sentence_1, sentence_2, pipeline):
    embedding_1 = pipeline(sentence_1)
    embedding_2 = pipeline(sentence_2)
    # compute cosine similarity between two sentences
    return torch.nn.functional.cosine_similarity(embedding_1, embedding_2, dim=1)


def evaluate_stsb(example):
    default = compute_sentence_similarity(example["sentence1"], example["sentence2"], vanilla_emb)
    quantized = compute_sentence_similarity(example["sentence1"], example["sentence2"], q8_emb)
    return {
        "reference": (example["label"] - 1) / (5 - 1),  # rescale to [0,1]
        "default": float(default),
        "quantized": float(quantized),
    }


result = eval_dataset.map(evaluate_stsb)

Parameter 'function'=<function evaluate_stsb at 0x7f92780c7600> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [6]:
default_acc = metric.compute(predictions=result["default"], references=result["reference"])
quantized = metric.compute(predictions=result["quantized"], references=result["reference"])

print("vanilla model: pearson=", default_acc["pearson"])
print("quantized model: pearson=", quantized["pearson"])
print(
    "The quantized model achieves ",
    round(quantized["pearson"] / default_acc["pearson"], 2) * 100,
    "% accuracy of the fp32 model",
)

vanilla model: pearson= 0.869619439095004
quantized model: pearson= 0.869326218489249
The quantized model achieves  100.0 % accuracy of the fp32 model


## Compare performance of the baselein and INT8 models

In [7]:
# FP32 baseline model
!benchmark_app -m all-MiniLM-L6-v2/openvino_model.xml -shape "input_ids[1,384],attention_mask[1,384],token_type_ids[1,384]" -api sync -niter 200

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2024.5.0-16971-8a02b4c17bb
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2024.5.0-16971-8a02b4c17bb
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[ WARNING ] Performance hint was not explicitly specified in command line. Device(CPU) performance hint will be set to PerformanceMode.LATENCY.
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 10.87 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     input_ids (node: input_ids) : i64 / [...] / [?,?]
[ INFO ]     attention_mask (node: attention_mask) : i64 / [...] / [?,?]
[ INFO ]     token_type_ids (node: token_type_ids) : i64 / [...] / [?,?]
[ INFO ] Model outputs:
[ INFO ]     last_hidden_state (node: __module.encoder.layer.5.out

[Step 11/11] Dumping statistics report
[ INFO ] Execution Devices:['CPU']
[ INFO ] Count:            200 iterations
[ INFO ] Duration:         1988.82 ms
[ INFO ] Latency:
[ INFO ]    Median:        9.70 ms
[ INFO ]    Average:       9.77 ms
[ INFO ]    Min:           9.54 ms
[ INFO ]    Max:           11.35 ms
[ INFO ] Throughput:   100.56 FPS


In [8]:
# INT8 counterpart
!benchmark_app -m all-MiniLM-L6-v2_int8/openvino_model.xml -shape "input_ids[1,384],attention_mask[1,384],token_type_ids[1,384]" -api sync -niter 200

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[Step 1/11] Parsing and validating input arguments
[ INFO ] Parsing input parameters
[Step 2/11] Loading OpenVINO Runtime
[ INFO ] OpenVINO:
[ INFO ] Build ................................. 2024.5.0-16971-8a02b4c17bb
[ INFO ] 
[ INFO ] Device info:
[ INFO ] CPU
[ INFO ] Build ................................. 2024.5.0-16971-8a02b4c17bb
[ INFO ] 
[ INFO ] 
[Step 3/11] Setting device configuration
[ WARNING ] Performance hint was not explicitly specified in command line. Device(CPU) performance hint will be set to PerformanceMode.LATENCY.
[Step 4/11] Reading model files
[ INFO ] Loading model files
[ INFO ] Read model took 15.46 ms
[ INFO ] Original model I/O parameters:
[ INFO ] Model inputs:
[ INFO ]     input_ids (node: input_ids) : i64 / [...] / [?,?]
[ INFO ]     attention_mask (node: attention_mask) : i64 / [...] / [?,?]
[ INFO ]     token_type_ids (node: token_type_ids) : i64 / [...] / [?,?]
[ INFO ] Model outputs:
[ INFO ]     last_hidden_state (node: __module.encoder.layer.5.out